# Plateforme Agnostique de Traitement et d'Analyse des Textes
### Carnet d'expérimentation
---

## Sujet : Identification des mots collés

---


# Observations et environnement
---

In [1]:
cd ../..

/Users/fm/Desktop/Work/Patat


In [2]:
import importlib
import pandas as pd
import seaborn as sns

## Paramètres globaux

In [3]:
random_state = 42

## Données 

In [4]:
import patat.db.article_db

In [5]:
article_db = patat.db.article_db.ArticleDb()

/Users/fm/Desktop/Work/Patat/patat/db/article_db.py:35: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [6]:
article_db._df['text'] = article_db._df['title']+'\n'+article_db._df['article']

In [7]:
df_texts = article_db._df

In [8]:
df_texts = df_texts[df_texts['text'].notna()]

In [9]:
df_texts.shape

(86803, 11)

In [10]:
df_texts = df_texts[df_texts['site']!='www.legorafi.fr']

In [11]:
df_texts.shape

(82315, 11)

In [12]:
### Preprocesor

import patat.util.text

preprocessor = patat.util.text.preprocess

### Tokenizer

from patat.ml.lex_analyser import LexAnalyser

lex = LexAnalyser()

tokenizer = lex.get_words
#tokenizer = lex.get_lemmas_from_text

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

# df_texts['text']=df_texts['text'].apply(preprocessor)

In [14]:
%%time
vectorizer = CountVectorizer(lowercase=True, preprocessor=preprocessor, tokenizer=tokenizer)
count_matrix = vectorizer.fit_transform(df_texts['text'])

count_array = count_matrix.toarray()
df_count = pd.DataFrame(data=count_array,columns = vectorizer.get_feature_names_out())

vocab_full = list(df_count.columns)

len(vocab_full)

CPU times: user 28.6 s, sys: 14.5 s, total: 43.1 s
Wall time: 56.1 s


404196

In [15]:
df_words = pd.DataFrame(vocab_full,columns=['word'])

In [16]:
df_words['c_count']=df_words['word'].apply(len)

In [17]:
df_words.sort_values('c_count',ascending=False)

,word,c_count
373153,unisindeindonésieirakiranirlandeislandeisraëli...,398
193305,jqyegayrqogbbgaaqaiaaiaigckaaiaadghabqqagaaaaa...,166
324064,saouditeargentinearméniearubaaustralieautriche...,142
186117,iqewgayiavaaogkaaaaoijbaaahcalaaaeaqaaaaccmaea...,140
119590,encéphaliteméningiteencéphalopathieconvulsions...,102
...,...,...
354820,tc,2
47701,br,2
47672,bq,2
354777,tb,2


In [18]:
word = df_words.loc[195889,'word']

In [19]:
word='aaaaaaaaaaaaaaaaaaaaaaaaaaaaaah'

In [20]:
df_count[df_count[word]!=0][word]

KeyError: 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaah'

In [ ]:
text = df_texts.loc[idx]['text']

In [ ]:
tokenizer(preprocessor(text))

In [ ]:
def find_word(word):
    return df_texts[df_texts['text'].str.contains(word)]

In [ ]:
df_words['c_count'].plot()

In [ ]:
df_words[df_words['c_count']<20]

In [ ]:
find_word('aaaaaaaaaaaa')

In [ ]:
df_texts.loc[50547]['text']

In [ ]:
df_texts_recueil = pd.read_csv('data/demo/221123-TextInfox.csv')

In [ ]:
df_texts_recueil

In [ ]:
df_texts_afp =  pd.read_csv('data/tmp/221118-TextsAfp.csv')

In [ ]:
df_texts_afp

In [ ]:
with open('data/tmp/221110-IgnoreLemmas.txt','r') as file:
    ignore_words = file.read().split()

In [ ]:
print(ignore_words[:100])
print(len(ignore_words))

In [ ]:
df_words = pd.read_csv('data/prod/221125-InfoxWords.csv',index_col=0)

In [ ]:
df_words

In [ ]:
ignore_words_small = list(df_words[df_words['common_80']].index)+list(df_words[df_words['rare_1']].index)
ignore_words_medium = list(df_words[df_words['common_80']].index)+list(df_words[df_words['rare_2']].index)
ignore_words_large = list(df_words[df_words['common_200']].index)+list(df_words[df_words['rare_5']].index)

In [ ]:
common_80 = list(df_words[df_words['common_80']].index)
common_200 = list(df_words[df_words['common_200']].index)

In [ ]:
ignore_words_small.sort

In [ ]:
len(ignore_words)

# Experience
---

## Feature Engineering

### Preprocessor

In [ ]:
import patat.util.text

importlib.reload(patat.util.text)
preprocessor = patat.util.text.preprocess
#preprocessor=None

### Tokenizer

In [ ]:
from patat.ml.lex_analyser import LexAnalyser

lex = LexAnalyser()

tokenizer = lex.get_lemmas_from_text

### TfIdfVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#afp_size = 3000
#afp_size = 120
afp_size = 20
#afp_size = 0
df_ml = pd.concat([df_texts_recueil,df_texts_afp.sample(afp_size,random_state=random_state)])

In [ ]:
%%time
vectorizer = TfidfVectorizer(lowercase=True, preprocessor=preprocessor, tokenizer=tokenizer, stop_words=[])
#vectorizer = CountVectorizer(lowercase=True, preprocessor=preprocessor, tokenizer=tokenizer, stop_words=ignore_words_small)
count_matrix = vectorizer.fit_transform(df_ml['text'])
count_array = count_matrix.toarray()
df_tf = pd.DataFrame(data=count_array,columns = vectorizer.get_feature_names_out())
df_tf.shape

df_X = df_tf
X = df_X

len(df_X.keys())

### Dimension reduction

## Entrainement Sklearn

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn import metrics

In [ ]:
#X = df_X
y=df_ml['infox']

In [ ]:
df_X.shape

### Initialisations

In [ ]:
train_size=0.8
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=random_state,train_size=train_size)

In [ ]:
def print_metrics(y_train, y_test, y_pred):
    print(f'Train size \t{len(y_train)} = {int(len(y_train)-y_train.sum())} False + {int(y_train.sum())} True' )
    print(f'Test size \t{len(y_test)} = {int(len(y_test)-y_test.sum())} False + {int(y_test.sum())} True\n' )
    print(f'Accuracy score \t{metrics.accuracy_score(y_test, y_pred)*100:.2f}%')
    print(f'Recall score \t{metrics.recall_score(y_test, y_pred)*100:.2f}%')
    print(f'F1 score \t{metrics.f1_score(y_test, y_pred)*100:.2f}%')
    cnf_matrix = metrics.confusion_matrix(y_test, y_pred,normalize='all')
    print(f'False Positive \t{cnf_matrix[0,1]*100:.2f}%')
    print(f'False Negative \t{cnf_matrix[1,0]*100:.2f}%\n')    

### Dummy Classifier

In [ ]:
from sklearn.dummy import DummyClassifier
#strategies = ['most_frequent', 'prior', 'stratified', 'uniform']
strategies = ['stratified', 'uniform']
for strategy in strategies:
    print('---------------')
    print(f'Strategy : {strategy}\n')
    classifier = DummyClassifier(strategy=strategy, random_state=random_state)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    print_metrics(y_train, y_test, y_pred)

### Confusion Matrix
||Pred 0|Pred 1|
|---|---|---|
|**Test 0**|True Negative|False Positive|
|**Test 1**|False Negative|True Positive|

### LogisticRegression

In [ ]:
max_iter=500
#C=0.40
C=100
solver = 'lbfgs'

In [ ]:
%%time
classifier = LogisticRegression(C=C, random_state=random_state, solver=solver, max_iter=max_iter)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(classifier)
print()
print_metrics(y_train, y_test, y_pred)

In [ ]:
# Cross validation
max_iter=500
#C=0.40
C=1000
solver = 'lbfgs'
classifier = LogisticRegression(C=C, random_state=random_state, solver=solver, max_iter=max_iter)
scoring = ['accuracy','recall','f1','roc_auc']
scores = cross_validate(classifier, X, y, cv=5,scoring=scoring)
for score in scores:
    print(score)
    print(scores[score])
    print(f'Mean : {scores[score].mean()} - Deviation : {scores[score].std()}')
#    print(f'Mean : {scores[score].mean()*100:.2f}%')
#    print(f'Deviation : {scores[score].std()*100:.2f}%')


### MLPClassifier

In [ ]:
alpha=1e-4
hidden_layer_sizes=(8,8)
solver = 'adam'
solvers = ['lbfgs', 'sgd', 'adam']


In [ ]:
%%time

classifier = MLPClassifier(solver=solver, alpha=alpha, hidden_layer_sizes=hidden_layer_sizes, random_state=random_state)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(classifier)
print()
print_metrics(y_train, y_test, y_pred)

In [ ]:
%%time
classifier = MLPClassifier(solver=solver, alpha=alpha, hidden_layer_sizes=hidden_layer_sizes, random_state=random_state)
scoring = ['accuracy','recall','f1','roc_auc']
scores = cross_validate(classifier, X, y, cv=5,scoring=scoring)
for score in scores:
    print(score)
    print(scores[score])
    print(f'Mean : {scores[score].mean()} - Deviation : {scores[score].std()}')
#    print(f'Mean : {scores[score].mean()*100:.2f}%')
#    print(f'Deviation : {scores[score].std()*100:.2f}%')

## Keras

In [ ]:
import tensorflow as tf
tf.keras.utils.set_random_seed(random_state)

In [ ]:
learn_size=0.90
X_learn,X_valid,y_learn,y_valid = train_test_split(X_train,y_train,random_state=random_state,train_size=learn_size)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(256, activation='relu', input_shape=[X.shape[1]]),
    layers.Dense(64, activation='relu'),    
    layers.GaussianDropout(0.40, seed=random_state),
    layers.Dense(1, activation='sigmoid'),
])

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Input(X.shape[1]),    
    layers.Dropout(0.40, seed=random_state),
    layers.Dense(8, activation='relu'),
#    layers.Dense(8, activation='relu'),    
    layers.Dropout(0.40, seed=random_state),
#    layers.Dense(8, activation='relu'),
#    layers.Dropout(0.40, seed=random_state),
    layers.Dense(1, activation='sigmoid'),
])

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
#    metrics=['binary_accuracy'],
    metrics=[tf.keras.metrics.AUC()],
    
)

In [ ]:
%%time
epoch = 10

early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.0001,
    restore_best_weights=True,
)

history = model.fit(
#    X_train.to_numpy(), y_train.to_numpy(),
#    validation_split=0.2,
    X_learn.to_numpy(), y_learn.to_numpy(),
    validation_data=(X_valid.to_numpy(), y_valid.to_numpy()),
    batch_size=24,
    epochs=epoch,
#    callbacks=[early_stopping],
    shuffle=True,
)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot(title="Cross-entropy")
history_df.loc[:, ['auc_1', 'val_auc_1']].plot(title="AUC")#history_df.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot(title="Accuracy")
#history_df.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot(title="Accuracy")#history_df.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot(title="Accuracy")








In [ ]:
history_df

In [ ]:
y_pred = model.predict(X_test.to_numpy())
y_pred = y_pred.round()
print(model)
print()
print_metrics(y_train, y_test, y_pred)

In [ ]:
model.summary()

# Sauvegarde des résultats
---

# Conclusions
---

- 

# Bricolages
---

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
tf.config.list_physical_devices()

In [ ]:
tf.device("/CPU:0")

In [ ]:
tf.debugging.set_log_device_placement(True)

# Place tensors on the CPU
with tf.device('/GPU:0'):
  a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

# Run on the GPU
c = tf.matmul(a, b)
print(c)

In [ ]:
tf.debugging.set_log_device_placement(True)

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
df_words['word']=df_words.index

In [ ]:
df_words['c_count']=df_words['word'].apply(len)

In [ ]:
df_words.sort_values('c_count',ascending=False).head(30)

In [ ]:
df_